In [ ]:
import pandas as pd
import numpy as np
import openpyxl
import io
import requests

In [ ]:
url = 
download = requests.get(url).content

In [ ]:
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

In [ ]:
df.head()

,Display Order Code,Invoice Created,Item SKU Code,Channel Name,Sale Order Status,Sale Order Item Status,Shipping Package Creation Date,Shipping Package Status Code,Tracking Number,Facility,Created,Channel Shipping,Item Details
0,91012630654,NaN,8906087770671,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:25:02,CREATED,NaN,SRF Pataudi,2023-01-11 21:12:45,False,NaN
1,91012630654,NaN,8906087770671,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:25:02,CREATED,NaN,SRF Pataudi,2023-01-11 21:12:45,False,NaN
2,91012630654,NaN,8906087771777,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:25:02,CREATED,NaN,SRF Pataudi,2023-01-11 21:12:45,False,NaN
3,91012630654,NaN,8906087772125,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:25:02,CREATED,NaN,SRF Pataudi,2023-01-11 21:12:45,False,NaN
4,91012630664,NaN,8906087773016,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:25:02,CREATED,NaN,SRF Pataudi,2023-01-11 21:12:45,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23732,91012675009,NaN,8906087771340,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN
23733,91012675016,NaN,8906087771883,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN
23734,91012675016,NaN,8906087771883,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN
23735,91012675016,NaN,8906087771883,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN


In [ ]:
df.rename(columns={"Display Order Code":"order_id","Item SKU Code":"sku"},inplace=True)
df['sku']=df['sku'].astype('category')
label1 = dict(enumerate(df['sku'].cat.categories))
df['sku']=df['sku'].astype('category').cat.codes
df.sort_values(by=["order_id","sku"],inplace=True)

In [ ]:
#combo data vs frequency
df_a=df[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df_a['combo']=df[['order_id','sku']].groupby('order_id')['sku'].apply(list).reset_index(name='combo')['combo']
df_a['combo_dist_tuple']=df[['order_id','sku']].groupby('order_id')['sku'].unique().apply(tuple).reset_index(name='combo_dt')['combo_dt']
tuple_qty=[]
for i in df_a['combo_dist_tuple']:
    len_tuple=len(i)
    tuple_qty.append(len_tuple)
df_a['len_tuple']=tuple_qty   

In [ ]:
df_a

,order_id,qty,combo,combo_dist_tuple,len_tuple
0,91012628671,2,"[133, 133]","(133,)",1
1,91012628684,6,"[110, 110, 110, 110, 129, 129]","(110, 129)",2
2,91012628819,2,"[110, 110]","(110,)",1
3,91012628823,2,"[28, 264]","(28, 264)",2
4,91012628829,4,"[10, 116, 118, 121]","(10, 116, 118, 121)",4
...,...,...,...,...,...
6661,91012675000,2,"[177, 179]","(177, 179)",2
6662,91012675003,2,"[41, 169]","(41, 169)",2
6663,91012675004,10,"[97, 97, 97, 97, 140, 140, 140, 140, 140, 140]","(97, 140)",2
6664,91012675009,12,"[105, 105, 105, 105, 105, 105, 105, 105, 105, ...","(105,)",1


In [ ]:
df_a.qty.mean()

3.560906090609061

In [ ]:
df_b=df_a.groupby('combo_dist_tuple').size().reset_index(name='freq').sort_values('freq',ascending=False)
df_b['order_list']=df_a.groupby('combo_dist_tuple')['order_id'].apply(list).reset_index(name='a')['a']
df_b.head(10)

,combo_dist_tuple,freq,order_list
2338,"(116,)",274,"[91012630743, 91012631434, 91012631518, 910126..."
2207,"(110,)",262,"[91012628819, 91012630701, 91012630760, 910126..."
1796,"(97, 140)",199,"[91012629928, 91012630707, 91012630991, 910126..."
1730,"(97,)",175,"[91012630321, 91012631013, 91012631496, 910126..."
2640,"(133,)",127,"[91012628671, 91012631115, 91012631161, 910126..."
2745,"(140,)",123,"[91012630969, 91012631834, 91012631853, 910126..."
2485,"(125,)",70,"[91012630838, 91012630906, 91012632897, 910126..."
1674,"(96,)",53,"[91012630968, 91012636624, 91012637275, 910126..."
2227,"(110, 116)",48,"[91012630738, 91012630956, 91012631827, 910126..."
1615,"(95,)",42,"[91012630739, 91012630769, 91012639977, 910126..."


In [ ]:
min_freq=25 ##select min frequency which you want
result_combo=df_b[df_b['freq']>=min_freq]  

In [ ]:
result_combo

,combo_dist_tuple,freq,order_list
2338,"(116,)",274,"[91012630743, 91012631434, 91012631518, 910126..."
2207,"(110,)",262,"[91012628819, 91012630701, 91012630760, 910126..."
1796,"(97, 140)",199,"[91012629928, 91012630707, 91012630991, 910126..."
1730,"(97,)",175,"[91012630321, 91012631013, 91012631496, 910126..."
2640,"(133,)",127,"[91012628671, 91012631115, 91012631161, 910126..."
2745,"(140,)",123,"[91012630969, 91012631834, 91012631853, 910126..."
2485,"(125,)",70,"[91012630838, 91012630906, 91012632897, 910126..."
1674,"(96,)",53,"[91012630968, 91012636624, 91012637275, 910126..."
2227,"(110, 116)",48,"[91012630738, 91012630956, 91012631827, 910126..."
1615,"(95,)",42,"[91012630739, 91012630769, 91012639977, 910126..."


In [ ]:
result_combo.freq.sum()

1764

In [ ]:
result_combo.to_excel(r"combo_result.xlsx")

In [ ]:
#now finding max order containing SKU pool

#preparing rest of the data after removing combo orders ids

df_rest=df_b[df_b['freq']<min_freq]

rest_orders=[]
for i in df_rest.order_list:
    rest_orders.extend(i)


df_rest=df[df['order_id'].isin(rest_orders)]
df_a=df_a[df_a['order_id'].isin(rest_orders)]

In [ ]:
df_rest.head()

,order_id,Invoice Created,sku,Channel Name,Sale Order Status,Sale Order Item Status,Shipping Package Creation Date,Shipping Package Status Code,Tracking Number,Facility,Created,Channel Shipping,Item Details
1117,91012628684,NaN,110,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:05,CREATED,NaN,SRF Pataudi,2023-01-11 22:08:19,False,NaN
1118,91012628684,NaN,110,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:05,CREATED,NaN,SRF Pataudi,2023-01-11 22:08:19,False,NaN
1119,91012628684,NaN,110,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:05,CREATED,NaN,SRF Pataudi,2023-01-11 22:08:19,False,NaN
1120,91012628684,NaN,110,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:05,CREATED,NaN,SRF Pataudi,2023-01-11 22:08:19,False,NaN
1121,91012628684,NaN,129,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:05,CREATED,NaN,SRF Pataudi,2023-01-11 22:08:19,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23728,91012675009,NaN,105,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN
23729,91012675009,NaN,105,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN
23730,91012675009,NaN,105,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN
23731,91012675009,NaN,105,MamaEarth-Production,PROCESSING,FULFILLABLE,2023-01-12 10:35:51,CREATED,NaN,SRF Pataudi,2023-01-12 10:22:24,False,NaN


In [ ]:
######## Below code is to find SKUs pool #########

In [ ]:
###select number of SKUs out of which combos will be created,these are most repetetive SKUs.

n=25   ##POOL STRENGTH

df_c=df_rest[['sku']].groupby('sku').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
topn=df_c.sort_values(by=['count'], ascending=False).head(n)['sku'].astype(np.int64).to_list()
topn.sort()

In [ ]:
df_c.head(5)

,sku,count
108,108,699
122,122,640
97,97,594
134,134,476
110,110,451


In [ ]:
percent_covered=df_c['count'][:n].sum()/df_c['count'].sum()
percent_covered

0.4839236739219233

In [ ]:
################################ Combo formation ###################################

combo_size=3

if combo_size==3:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                combo[0]=topn[i]
                combo[1]=topn[j]
                combo[2]=topn[k]
                a.append(combo.copy())

elif combo_size==4:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    combo[0]=topn[i]
                    combo[1]=topn[j]
                    combo[2]=topn[k]
                    combo[3]=topn[l]
                    a.append(combo.copy())
                   
elif combo_size==5:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        combo[0]=topn[i]
                        combo[1]=topn[j]
                        combo[2]=topn[k]
                        combo[3]=topn[l]
                        combo[4]=topn[m]
                        a.append(combo.copy())
    

In [ ]:
len(a)

4060

In [ ]:
result_df=pd.DataFrame(columns=['combo',"freq"])
result_df['combo']=a


In [ ]:
## Searching number of fully inclusive orders in a given pair of SKUs

df3=df_rest[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df3['sku_list']=df_rest[['order_id','sku']].groupby('order_id')['sku'].unique().apply(list).reset_index(name='combo')['combo']

length_orders=df3['order_id'].count() 

res_list=[]
for sample in result_df['combo']:
    mfg=0
    for j in range(length_orders):
        test_list=df3.sku_list[j]
        check=all(item in sample for item in test_list) 
        if check is True:
            mfg=mfg+1
        else:
            mfg=mfg
    res_list.append(mfg)
        
result_df['freq']=res_list
         


In [ ]:
result_df.sort_values("freq",ascending=False).head(8)

,combo,freq
809,"[53, 54, 297]",75
786,"[53, 54, 75]",74
805,"[53, 54, 169]",72
2282,"[75, 169, 297]",70
2229,"[75, 126, 169]",69
406,"[41, 53, 54]",69
724,"[41, 126, 169]",68
526,"[41, 75, 169]",67


In [ ]:
combo=[58,60,117]

In [ ]:
df_x=df_a[['order_id','combo']]
a=[]
for sample in df_x.combo:
    test=all(item in combo for item in sample)
    if test is True:
        a.append('yes')
    else:
        a.append('no')
df_x['order_in_pool']=a 

C:\Users\Amit Singh\AppData\Local\Temp\ipykernel_22256\3342354665.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=a


In [ ]:
df_x[df_x['order_in_pool']=='yes'].count()


order_id         35
combo            35
order_in_pool    35
dtype: int64

In [ ]:
df_x[df_x['order_in_pool']=='yes'].to_excel(r'rest2.xlsx')

In [ ]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]

In [ ]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]

n=30 ##POOL STRENGTH
df_c=df_rest[['sku']].groupby('sku').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
topn=df_c.sort_values(by=['count'], ascending=False).head(n)['sku'].astype(np.int64).to_list()
topn.sort()

################################ Combo formation ###################################

combo_size=3

if combo_size==2:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
                combo[0]=topn[i]
                combo[1]=topn[j]
                a.append(combo.copy())
                         

elif combo_size==3:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                combo[0]=topn[i]
                combo[1]=topn[j]
                combo[2]=topn[k]
                a.append(combo.copy())

elif combo_size==4:    
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    combo[0]=topn[i]
                    combo[1]=topn[j]
                    combo[2]=topn[k]
                    combo[3]=topn[l]
                    a.append(combo.copy())
                   
elif combo_size==5:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        combo[0]=topn[i]
                        combo[1]=topn[j]
                        combo[2]=topn[k]
                        combo[3]=topn[l]
                        combo[4]=topn[m]
                        a.append(combo.copy())
elif combo_size==6:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        for n in range(m+1,len(topn)):
                            combo[0]=topn[i]
                            combo[1]=topn[j]
                            combo[2]=topn[k]
                            combo[3]=topn[l]
                            combo[4]=topn[m]
                            combo[5]=topn[n]
                            a.append(combo.copy())
elif combo_size==8:
    a=[]
    combo=[None]*combo_size
    for i in range(len(topn)):
        for j in range(i+1,len(topn)):
            for k in range(j+1,len(topn)):
                for l in range(k+1,len(topn)):
                    for m in range(l+1,len(topn)):
                        for n in range(m+1,len(topn)):
                            for o in range(n+1,len(topn)):
                                for p in range(o+1,len(topn)):
                                    combo[0]=topn[i]
                                    combo[1]=topn[j]
                                    combo[2]=topn[k]
                                    combo[3]=topn[l]
                                    combo[4]=topn[m]
                                    combo[5]=topn[n]
                                    combo[6]=topn[o]
                                    combo[7]=topn[p]
                                    a.append(combo.copy()) 

result_df1=pd.DataFrame(columns=['combo',"freq"])
result_df1['combo']=a
##------------------------------------------------------------------------------------------
df3=df_rest[['order_id','sku']].groupby('order_id').size().reset_index(name="qty")
df3['sku_list']=df_rest[['order_id','sku']].groupby('order_id')['sku'].unique().apply(list).reset_index(name='combo')['combo']

##-------------------------------------------------------------------------------------------
length_orders=df3['order_id'].count() 

res_list=[]
for sample in result_df1['combo']:
    mfg=0
    for j in range(length_orders):
        test_list=df3.sku_list[j]
        check=all(item in sample for item in test_list) 
        if check is True:
            mfg=mfg+1
        else:
            mfg=mfg
    res_list.append(mfg)
        
result_df1['freq']=res_list
result_df1.sort_values("freq",ascending=False).head(8)

In [ ]:
combo=[9,34,64]

In [ ]:
df_x=df_a[['order_id','combo']]
a=[]
for sample in df_x.combo:
    test=all(item in combo for item in sample)
    if test is True:
        a.append('yes')
    else:
        a.append('no')
df_x['order_in_pool']=a

C:\Users\Amit Singh\AppData\Local\Temp\ipykernel_22256\3208110827.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=a


In [ ]:
df_x[df_x['order_in_pool']=='yes'].count()

order_id         20
combo            20
order_in_pool    20
dtype: int64

In [ ]:
df_x[df_x['order_in_pool']=='yes'].to_excel(r'rest1.xlsx')

In [ ]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]

In [ ]:
df_a

,order_id,qty,combo,combo_dist_tuple,len_tuple
1,1842704,2,"[16, 110]","(16, 110)",2
2,1855474,3,"[47, 49, 117]","(47, 49, 117)",3
3,1855562,5,"[41, 41, 57, 60, 117]","(41, 57, 60, 117)",4
4,1855606,5,"[41, 47, 51, 84, 117]","(41, 47, 51, 84, 117)",5
5,1855613,3,"[41, 82, 117]","(41, 82, 117)",3
...,...,...,...,...,...
1889,SO52493,1,[38],"(38,)",1
1890,SO52494,1,[34],"(34,)",1
1891,SO52495,1,[32],"(32,)",1
1892,SO52496,1,[65],"(65,)",1


In [ ]:
df_rest

,order_id,Invoice Created,sku,Channel Name,Sale Order Status,Sale Order Item Status,Shipping Package Status Code,Tracking Number,Dispatch Date,Facility,Created,Channel Shipping,Item Details
0,1842704,NaN,16,SHOPIFY,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-04 11:01:03,False,NaN
1,1842704,NaN,110,SHOPIFY,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-04 11:01:03,False,NaN
2,1855474,NaN,47,SHOPIFY,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-05 16:20:16,False,NaN
3,1855474,NaN,49,SHOPIFY,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-05 16:20:16,False,NaN
4,1855474,NaN,117,SHOPIFY,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-05 16:20:16,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4743,SO52493,NaN,38,Compensation,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-06 22:51:29,False,NaN
4801,SO52494,NaN,34,Compensation,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-06 22:52:39,False,NaN
4802,SO52495,NaN,32,Compensation,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-06 22:53:57,False,NaN
4803,SO52496,NaN,65,Compensation,PROCESSING,FULFILLABLE,CREATED,NaN,NaN,mCaff_Mumbai3_SR2,2022-10-06 22:55:05,False,NaN


In [ ]:
n=10 ##POOL STRENGTH
df_c=df_rest[['sku']].groupby('sku').size().reset_index(name='count').sort_values(by=['count'], ascending=False)
topn=df_c.sort_values(by=['count'], ascending=False).head(n)['sku'].astype(np.int64).to_list()
topn.sort()

In [ ]:
#topn=df_c['sku'][0:20].to_list()

In [ ]:
topn

[10, 11, 20, 25, 27, 28, 54, 67, 76, 78]

In [ ]:
df_x=df_a[['order_id','combo']]
a=[]
for sample in df_x.combo:
    test=all(item in combo for item in sample)
    if test is True:
        a.append('yes')
    else:
        a.append('no')
df_x['order_in_pool']=a
df_x[df_x['order_in_pool']=='yes'].count()

C:\Users\Amit Singh\AppData\Local\Temp\ipykernel_16140\1651454237.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x['order_in_pool']=a


order_id         17
combo            17
order_in_pool    17
dtype: int64

In [ ]:
df_x[df_x['order_in_pool']=='yes'].to_excel(r'rest1.xlsx')

In [ ]:
df_med=df_x[df_x['order_in_pool']=='no']
exclude_list=df_med['order_id'].to_list()

df_rest=df[df['order_id'].isin(exclude_list)]
df_a=df_a[df_a['order_id'].isin(exclude_list)]